# ООП-2

### Глава 1: датаклассы и енамы

В датаклассе не нужен инит, достаточно просто объявит нужны в нем поля и дальше так и использовать

In [28]:
from dataclasses import dataclass

@dataclass
class User:
    email: str
    gender: int  # 1 for male, 0 for female
    first_name: str
    middle_name: str
    last_name: str

    @property
    def full_name(self):
        return f"{self.first_name} {self.middle_name} {self.last_name}"

In [33]:
import json


response = {
    "email": "johndoe@gmail.com",
    "gender": 1,
    "first_name": "John",
    "middle_name": "Sergeevich",
    "last_name": "Doe"
}

response = json.dumps(response)
response

'{"email": "johndoe@gmail.com", "gender": 1, "first_name": "John", "middle_name": "Sergeevich", "last_name": "Doe"}'

In [35]:
user = User(**json.loads(response))
print(user.email)
print(user.full_name)

johndoe@gmail.com
John Sergeevich Doe


In [36]:
user = User("johndoe@gmail.com", 1, "John", "Sergeevich", "Doe")
print(user.email)
print(user.full_name)

johndoe@gmail.com
John Sergeevich Doe


Из грустного: gender сейчас хранится числом и получается какая-то магия, постоянно надо помнить, какому полу соответствует какое число

In [49]:
from dataclasses import dataclass
from enum import Enum, auto


class Gender(Enum):  # тут также можно использовать IntEnum
    female = auto()  # auto автоматически пронумерует элементы енама
    male = auto()  # но можно задать какое-то и свое значение, например
                   # male = "male"
                   # female = "female"

In [50]:
print(Gender.female)
print(Gender.female.name)
print(Gender.female.value)

Gender.female
female
1


In [51]:
print(Gender.male)
print(Gender.male.name)
print(Gender.male.value)

Gender.male
male
2


In [54]:
Gender._member_map_

{'female': <Gender.female: 1>, 'male': <Gender.male: 2>}

In [55]:
Gender._value2member_map_

{1: <Gender.female: 1>, 2: <Gender.male: 2>}

In [57]:
Gender.male

<Gender.male: 2>

In [58]:
from dataclasses import dataclass
from enum import Enum, auto


class Gender(Enum):  # тут также можно использовать IntEnum
    female = auto()
    male = auto()

    def __str__(self):
        return self.name  # name, чтобы получить названия самого элемента енама, value -- чтобы получить значение

    __repr__ = __str__

In [60]:
Gender.male

male

In [61]:
from dataclasses import dataclass

@dataclass
class User:
    email: str
    gender: Gender
    first_name: str
    middle_name: str
    last_name: str

    @property
    def full_name(self):
        return f"{self.first_name} {self.middle_name} {self.last_name}"

    @property
    def is_male(self):
        return self.gender == Gender.male

In [62]:
user = User(email="johndoe@gmail.com", gender=Gender.male, first_name="John", middle_name="Sergeevich", last_name="Doe")

print(user.email)
print(user.full_name)
print(user.gender)
print(user.is_male)

johndoe@gmail.com
John Sergeevich Doe
male
True


### Глава 2: классметоды и статикметоды

Classmethod используется когда нужно вызвать функцию класса, не создавая объект. Чаще всего это нужно для создания экземпляра класса на основе каких-то данных которые еще как-то нужно обработать

In [63]:
from dataclasses import dataclass
from datetime import date

@dataclass
class Person:
    age: int
    name: str

    @classmethod
    def from_birthyear(cls, name: str, birth_year: int) -> "Person":  # в cls лежит фактически сам класс
        return cls(
            age=date.today().year - birth_year,
            name=name,
        )

    def __str__(self):
        return f"{self.name}, age {self.age}"

In [65]:
person = Person.from_birthyear("John", 2000)

print(person)

John, age 23


Staticmethod же применяется обычно для инкапуляции какой-то логики внутри класса, которой вроде и не нужен self, а вроде и не хочется это просто отдельными функциями плодить

In [66]:
from dataclasses import dataclass
from datetime import date

@dataclass
class Person:
    age: int
    name: str

    @staticmethod
    def calculate_age(birth_year: int) -> int:  # заметьте, тут не нужен self/cls
        return date.today().year - birth_year

    @classmethod
    def from_birthyear(cls, name: str, birth_year: int) -> "Person":
        return cls(
            age=cls.calculate_age(birth_year),  # а вот тут уже нужен
            name=name,
        )

    def __str__(self):
        return f"{self.name}, age {self.age}"

In [67]:
person = Person.from_birthyear("John", 2000)

print(person)

John, age 23


### Глава 3: (не)безопасность

В питоне есть так называемые private переменные класса, но работают они с переменным успехом.

In [70]:
class Client:
    def __init__(self, id, phone_number):
        self._id = id
        self.__phone_number = phone_number

In [71]:
client = Client(123, "88005553535")
client._id

123

In [72]:
client.__phone_number

AttributeError: ignored

НО!!!

In [73]:
client.__dict__

{'_id': 123, '_Client__phone_number': '88005553535'}

In [74]:
client._Client__phone_number

'88005553535'

### Глава 4: namedtuple

Это некоторый гибрид кортежа и класса. Работает как обычный кортеж, только помимо этого у него именованные поля.

In [75]:
from collections import namedtuple

Point = namedtuple("Point", ["x", "y"])

w = Point(5, 6)

print(w)

Point(x=5, y=6)


In [77]:
w[0], w[1], w.x, w.y

(5, 6, 5, 6)

В остальном это обычный tuple:

In [78]:
for coord in w:
    print(coord)

5
6


In [79]:
w.count(5)

1

In [80]:
len(w)

2

In [82]:
from typing import NamedTuple

class Point(NamedTuple):
    x: int
    y: int

In [83]:
w = Point(5, 6)

print(w)

Point(x=5, y=6)


In [84]:
for coord in w:
    print(coord)

5
6


In [85]:
w.count(5)

1

# Abstract Base Classes

In [15]:
from abc import ABC, abstractmethod


class SmartWatchInterface(ABC):
    @abstractmethod
    def show_time(self):
        pass

    @abstractmethod
    def show_battery_level(self):
        pass

    def say_goodbye(self):
        print("Bye!")

In [16]:
import time
import random


class MySmartWatch(SmartWatchInterface):
    def show_time(self):
        print(time.time())

    def show_battery_level(self):
        print(random.randint(1, 100))

    def say_hello(self):
        print("Hello")

In [17]:
watch = MySmartWatch()

In [18]:
watch.show_time()
watch.show_battery_level()
watch.say_hello()
watch.say_goodbye()

1699451671.4272513
44
Hello
Bye!


In [19]:
from collections.abc import Container, Sequence, Callable

In [42]:
class MyContainer(Container):
    def __init__(self, *objs):
        self.objs = objs

    def __contains__(self, obj):
        print("in contains method")
        return obj in self.objs

In [43]:
my_container = MyContainer(1,2,3,4,5,6,7,8,9,10)

In [44]:
1 in my_container

in contains method


True

In [45]:
issubclass(MyContainer, Container)

True

In [46]:
isinstance(my_container, Container)

True

In [31]:
class MySequence(Sequence):
    def __init__(self, *items):
        self.items = items

    def __len__(self):
        return len(self.items)

    def __getitem__(self, item):
        return self.items[item]

In [32]:
my_sequence = MySequence(1,2,3,4,5,6,7,8,9)

In [33]:
len(my_sequence)

9

In [34]:
my_sequence[1]

2

In [37]:
class MyCallable(Callable):
    def __call__(self):
        print("I'm called!")

my_callable = MyCallable()
my_callable()

I'm called!
